In [161]:
from torchtext import data
import spacy
import visdom
import numpy as np

vis = visdom.Visdom()
nlp = spacy.load('es')

# Intro

En esta fase vamos a cargar los datos, procesarlos y visualizarlos.

Para la visualización vamos a utilizar Visdom para lo que no tenemos que olvidar lanzar el servidor de visualizaciones, ejecutando:
 
``` 
python -m visdom.server

```

# Funciones visualización

Hemos creado algunas funciones de visualización para ayudarnos a la hora de procesar los datos y tomar decisiones en la configuración de nuestras 'features' 
y parámetros del modelo predictivo.

In [162]:
def visualize_field(vocab, size, title='Distribution'):
    counts = []
    labels = []
    for i in range(size):
        word = vocab.itos[i]
        freq = vocab.freqs[word]
        counts.append(freq)
        labels.append(word)
    vis.bar(X=counts, opts=dict(rownames=labels, title=title)) #opts=dict(numbins=30)

def visualize_correlation(examples, vocab_text, vocab_labels, size, type='stacked', title='Correlation'):
    heats = [ [0] * len(label.vocab.itos) for i in range(size)]
    words = [vocab_text.itos[i] for i in range(size)]
    classes = [vocab_labels.itos[i] for i in range(len(vocab_labels.itos))]
    for example in examples:
        label_id = vocab_labels.stoi[example.label]
        text = example.text
        for token in text:
            i = vocab_text.stoi[token]
            if(i < size):
                heats[i][label_id] += 1
    if(type == 'stacked'):
        vis.bar(
            X=np.array(heats),
            opts=dict(
                stacked=True,
                title=title,
                legend=classes,
                rownames=words
            )
        )
    else:
        vis.heatmap(
            X=np.array(heats),
            opts=dict(
                columnnames=classes,
                rownames=words,
                title=title,
                colormap='Electric'
            )
        )

# Funciones de procesamiento de texto


Un aspecto clave ante cualquier problema de PLN es la manera en la que procesamos nuestros datos en crudo. Lo primero que tenemos que hacer es definir la manera en la que separamos el texto en los simbolos que contiene. Para ello vamos a definir una función de tokenización:

In [163]:
def tokenize_with_filter(filter):
    def tokenize(text):
        return [t.lower_ for t in nlp(text) if filter(t)]
    return tokenize

# Carga de datos

Vamos a definir los datos de entrenamiento y prueba, así como los campos que vamos a utilizar:

In [164]:
DATASET_PATH = '/Users/oeg/dev/recognai/data/datasets/cosetdata/'

# Primero definimos los campos del dataset de entrenamiento
f = (lambda t: not t.is_stop and t.is_alpha) # Examples: t.is_alpha, full documentation at:
twitter_id = data.Field()
text = data.Field(tokenize=tokenize_with_filter(f))
label = data.Field(sequential=False) # preprocessing=(lambda s: int(s))

train = data.TabularDataset(path= DATASET_PATH + 'coset-train.csv',
                            format='csv',
                            fields= [('id', twitter_id), ('text', text),('label',label)],
                            skip_header=True)

dev = data.TabularDataset(path= DATASET_PATH + 'coset-dev.csv',
                            format='csv',
                            fields= [('id', twitter_id), ('text', text),('label',label)],
                            skip_header=True)

# Creación de vocabularios

...

In [165]:
text.build_vocab(train)
label.build_vocab(train)

# Visualización

Vamos a visualizar nuestros datos.

Podemos visualizar:

* la distribución de palabras del texto de entrada (e.g., tweets)
* la distribución de las etiquetas o clases asignadas a cada texto de entrada
* o la correlación entre clases y palabras en el texto (correlation)



In [166]:
visualize_correlation(train.examples, 
                      text.vocab, 
                      label.vocab, 
                      50, 
                      'stacked', 
                      title='Train: Correlación entre clases y palabras')

visualize_correlation(train.examples, 
                      text.vocab, 
                      label.vocab, 
                      50, 
                      'heatmap', 
                      title='Train: Correlación entre clases y palabras')

visualize_field(text.vocab, 
                500, 
                title='Train: Distribución de palabras')

visualize_field(label.vocab, 
                len(label.vocab.itos), 
                title='Train: Distribución de clases')


AttributeError: 'Vocab' object has no attribute 'vocab'

In [ ]:
# Si queremos ver la distribución en el dev set, podemos volver a crear el vocabulario solo usando el dev set
text.build_vocab(dev)
label.build_vocab(dev)

visualize_correlation(dev.examples, 
                      text.vocab, 
                      label.vocab, 
                      50, 
                      'stacked', 
                      title='Dev: Correlación entre clases y palabras')

visualize_correlation(dev.examples, 
                      text.vocab, 
                      label.vocab, 
                      50, 
                      'heatmap', 
                      title='Dev: Correlación entre clases y palabras')

visualize_field(text.vocab, 500, title='Dev: Distribución de palabras')

visualize_field(label.vocab, len(label.vocab.itos), title='Dev: Distribución de clases')
print(label.vocab.itos)

Por ejemplo:
![title](../img/data-analysis.jpg)